# Eclipsing binary: Linear solution

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import starry
from scipy.linalg import cho_solve
from corner import corner

np.random.seed(12)
starry.config.lazy = False
starry.config.quiet = True

## Load the data

We generated the data in **EclipsingBinary_Generate.ipynb**. Let's load that dataset:

In [ ]:
# Run the Generate notebook if needed
if not os.path.exists("eb.npz"):
    import nbformat
    from nbconvert.preprocessors import ExecutePreprocessor

    with open("EclipsingBinary_Generate.ipynb") as f:
        nb = nbformat.read(f, as_version=4)
    ep = ExecutePreprocessor(timeout=600, kernel_name="python3")
    ep.preprocess(nb);

In [ ]:
data = np.load("eb.npz")
A = data["A"].item()
B = data["B"].item()
t = data["t"]
flux = data["flux"]
sigma = data["sigma"]

Instantiate the primary, secondary, and system objects. We assume we know the true values of all the orbital parameters and star properties, *except* for the two surface maps.

In [ ]:
# Primary
pri = starry.Primary(
    starry.Map(ydeg=A["ydeg"], udeg=A["udeg"], inc=A["inc"], amp=A["amp"]),
    r=A["r"],
    m=A["m"],
    prot=A["prot"],
)
pri.map[1:] = A["u"]

# Secondary
sec = starry.Secondary(
    starry.Map(ydeg=B["ydeg"], udeg=B["udeg"], inc=B["inc"], amp=B["amp"]),
    r=B["r"],
    m=B["m"],
    porb=B["porb"],
    prot=B["prot"],
    t0=B["t0"],
    inc=B["inc"],
)
sec.map[1:] = B["u"]

# System
sys = starry.System(pri, sec)

Here's the light curve we're going to do inference on:

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 5))
ax.plot(t, flux, "k.", alpha=0.5, ms=4)
ax.set_xlabel("time [days]", fontsize=24)
ax.set_ylabel("normalized flux", fontsize=24);

## Linear solve

In [ ]:
# HACK: Pre-compile the solve function
# to get an accurate timing test below!
sys.set_data(np.array([0.0]), C=1.0)
sys.solve(t=np.array([0.0]));

In order to compute the posterior over maps, we need a prior for the spherical harmonic coefficients of each star. The linear solve requires Gausssian priors on the spherical harmonic coefficients; these are specified in terms of a mean vector $\mu$ and a covariance matrix $\Lambda$. The easiest kind of prior we can place is an isotropic prior (no preferred direction), in which $\mu = 0$ and $\Lambda$ is a diagonal matrix. In this case, the diagonal entries of $\Lambda$ are related to the power spectrum of the map. We'll discuss this in more detail later, but for now let's assume a flat power spectrum, in which there is no preferred scale, so $\Lambda = \lambda I$. The quantity $\lambda$ is essentially a regularization parameter, whose amplitude controls the relative weighting of the data and the prior in determining the posterior. Let's choose $\lambda = 10^{-2}$ for definiteness, although readers are encouraged to experiment with different values.

In [ ]:
pri.map.set_prior(L=1e-2)
sec.map.set_prior(L=1e-2)

(Note that, by default, $\mu = 0$. Note also that $L$ may be a scalar, vector, or matrix, and `starry` will construct the covariance matrix for you. Alternatively, users may instead specify `cho_L`, the Cholesky factorization of the covariance matrix).

Next, we specify the data and data covariance $C$ (the measurement uncertainty):

In [ ]:
sys.set_data(flux, C=sigma ** 2)

(As before, users can pass a scalar, vector or matrix as the data covariance, or the Cholesky factorization `cho_C`).

Finally, let's solve the linear problem! We do this by calling `sys.solve()` and passing the array of times at which to evaluate the light curve. The method returns the mean $\mu$ and Cholesky factorization $L$ of the posterior covariance for each body in the system. Let's time how long this takes:

In [ ]:
%%time
mu, cho_cov = sys.solve(t=t)

The linear solve is **extremely fast**! Note that once we run the `solve` method, we can call the `draw` method from each map to draw samples from the posterior. Let's do that and visualize two random samples from each map:

In [ ]:
pri.map.draw()
pri.map.show(theta=np.linspace(0, 360, 50))

In [ ]:
sec.map.draw()
sec.map.show(theta=np.linspace(0, 360, 50))

We can compare these maps to the true maps:

In [ ]:
pri.map.draw()
pri_draw = pri.map.render(projection="rect")

pri.map[1:, :] = A["y"]
pri_true = pri.map.render(projection="rect")

sec.map.draw()
sec_draw = sec.map.render(projection="rect")

sec.map[1:, :] = B["y"]
sec_true = sec.map.render(projection="rect")

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(8, 4.5))
ax[0, 0].imshow(
    pri_true,
    origin="lower",
    extent=(-180, 180, -90, 90),
    cmap="plasma",
    vmin=0,
    vmax=0.4,
)
ax[1, 0].imshow(
    pri_draw,
    origin="lower",
    extent=(-180, 180, -90, 90),
    cmap="plasma",
    vmin=0,
    vmax=0.4,
)
ax[0, 1].imshow(
    sec_true,
    origin="lower",
    extent=(-180, 180, -90, 90),
    cmap="plasma",
    vmin=0,
    vmax=0.04,
)
ax[1, 1].imshow(
    sec_draw,
    origin="lower",
    extent=(-180, 180, -90, 90),
    cmap="plasma",
    vmin=0,
    vmax=0.04,
)
ax[0, 0].set_title("primary")
ax[0, 1].set_title("secondary")
ax[0, 0].set_ylabel("true", rotation=0, labelpad=20)
ax[1, 0].set_ylabel("draw", rotation=0, labelpad=20);

Not bad!

The other thing we can do is draw samples from this solution and plot the traditional corner plot for the posterior. Armed with the posterior mean `mu` and the Cholesky factorization of the covariance `cho_cov`, this is [super easy to do](https://en.wikipedia.org/wiki/Multivariate_normal_distribution#Drawing_values_from_the_distribution). Let's generate 10000 samples from the posterior of the primary's surface map:

In [ ]:
%%time
nsamples = 10000
u = np.random.randn(pri.map.Ny - 1, nsamples)
samples = mu[0].reshape(1, -1) + cho_solve((cho_cov[0], True), u).T

Here's the posterior for the first nine coefficients:

In [ ]:
fig, ax = plt.subplots(9, 9, figsize=(7, 7))
labels = [
    r"$Y_{%d,%d}$" % (l, m) for l in range(pri.map.ydeg + 1) for m in range(-l, l + 1)
]
corner(samples[:, :9], fig=fig, labels=labels)
for axis in ax.flatten():
    axis.xaxis.set_tick_params(labelsize=6)
    axis.yaxis.set_tick_params(labelsize=6)
    axis.xaxis.label.set_size(12)
    axis.yaxis.label.set_size(12)
    axis.xaxis.set_label_coords(0.5, -0.6)
    axis.yaxis.set_label_coords(-0.6, 0.5)